In [ ]:
!pip install keras

In [ ]:
!pip install gensim

In [ ]:
!pip install jieba==0.38

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.38-py3-none-any.whl size=7361437 sha256=9ec06fdb633092e0b6e34bfe1a08bd557b46de6bd622808614059e73620609fd
  Stored in directory: /root/.cache/pip/wheels/ed/12/bb/dafef5e2839e3b563b6cd37686caff82f364ac985fe0f1d757
Successfully built jieba
  Attempting uninstall: jieba
    Found existing installation: jieba 0.42.1
    Uninstalling jieba-0.42.1:
      Successfully uninstalled jieba-0.42.1


In [ ]:
from sklearn import linear_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

neg = pd.read_csv('/content/drive/MyDrive/iss/NLP Project/SentimentAnalysis-master/data/neg.csv', header=None, index_col=None)
pos = pd.read_csv('/content/drive/MyDrive/iss/NLP Project/SentimentAnalysis-master/data/pos.csv', header=None, index_col=None, on_bad_lines='skip')
neu = pd.read_csv('/content/drive/MyDrive/iss/NLP Project/SentimentAnalysis-master/data/neutral.csv', header=None, index_col=None)

In [ ]:
neu.columns

Index([0], dtype='int64')

In [ ]:
import numpy as np

combined = np.concatenate((pos[0], neu[0], neg[0]))
combined.shape

(21088,)

In [ ]:
# pos -> 1; neu -> 0; neg -> -1
y = np.concatenate((np.ones(len(pos), dtype=int), np.zeros(len(neu), dtype=int), -1*np.ones(len(neg),dtype=int)))
y.shape

(21088,)

In [ ]:
import jieba

#对句子经行分词，并去掉换行符
def tokenizer(text):
    ''' Simple Parser converting each document to lower-case, then
        removing the breaks for new lines and finally splitting on the
        whitespace
    '''
    text = [jieba.lcut(document.replace('\n', '')) for document in text]
    return text

combined = tokenizer(combined)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.928 seconds.
DEBUG:jieba:Loading model cost 0.928 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence
import multiprocessing

cpu_count = multiprocessing.cpu_count() # 4
vocab_dim = 100
n_iterations = 10  # ideally more..   epoch
n_exposures = 10 # 所有频数超过10的词语
window_size = 7
n_epoch = 4
input_length = 100
maxlen = 100

def create_dictionaries(model=None, combined=None):
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        # 使用 model.wv.key_to_index 替代 model.vocab.keys()
        gensim_dict.doc2bow(model.wv.key_to_index.keys(),
                            allow_update=True)
        # 由于 model.wv.key_to_index 已经是 {word: index} 的映射，所以可以直接使用
        w2indx = {word: index + 1 for word, index in model.wv.key_to_index.items()}  # 词语的索引
        w2vec = {word: model.wv[word] for word in w2indx.keys()}  # 词语的词向量

        def parse_dataset(combined):  # 闭包-->临时使用
            data = []
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0)  # 词频小于10的词语索引为0
                data.append(new_txt)
            return data  # word => index

        combined = parse_dataset(combined)
        combined = sequence.pad_sequences(combined, maxlen=maxlen)  # 对句子进行填充
        return w2indx, w2vec, combined
    else:
        print('No data provided...')


#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def word2vec_train(combined):

    model = Word2Vec(vector_size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     epochs=n_iterations)
    model.build_vocab(combined) # input: list
    #model.train(combined)
    model.train(combined, total_examples=model.corpus_count, epochs=model.epochs)
    model.save('/content/drive/MyDrive/iss/NLP Project/SentimentAnalysis-master/model/Word2vec_model.pkl')
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined

print ('Training a Word2vec model...')
index_dict, word_vectors,combined=word2vec_train(combined)

Training a Word2vec model...


In [ ]:
from sklearn.model_selection import train_test_split  # 更新导入路径
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Activation  # 更新了导入的层
from keras.models import model_from_yaml
import numpy as np
import sys
sys.setrecursionlimit(1000000)
import yaml
import keras

batch_size = 32


def get_data(index_dict,word_vectors,combined,y):

    n_symbols = len(index_dict) + 1  # 所有单词的索引数，频数小于10的词语索引为0，所以加1
    embedding_weights = np.zeros((n_symbols, vocab_dim)) # 初始化 索引为0的词语，词向量全为0
    for word, index in index_dict.items(): # 从索引为1的词语开始，对每个词语对应其词向量
        embedding_weights[index, :] = word_vectors[word]
    x_train, x_test, y_train, y_test = train_test_split(combined, y, test_size=0.2)
    y_train = keras.utils.to_categorical(y_train,num_classes=3)
    y_test = keras.utils.to_categorical(y_test,num_classes=3)
    # print x_train.shape,y_train.shape
    return n_symbols,embedding_weights,x_train,y_train,x_test,y_test


##定义网络结构
def train_lstm(n_symbols, embedding_weights, x_train, y_train, x_test, y_test):
    print('Defining a Simple Keras Model...')
    model = Sequential()
    model.add(Embedding(output_dim=vocab_dim,
                        input_dim=n_symbols,
                        mask_zero=True,
                        weights=[embedding_weights],
                        input_length=input_length))
    model.add(LSTM(units=50, activation='tanh', recurrent_activation='hard_sigmoid'))  # 更新参数名
    model.add(Dropout(0.5))
    model.add(Dense(units=3, activation='softmax'))  # 明确指定 units 参数
    # Activation 层在这里可能是多余的，因为上一层 Dense 已经使用了 'softmax' 激活函数

    print('Compiling the Model...')
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    print("Train...")
    model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epoch, verbose=1)

    print("Evaluate...")
    score = model.evaluate(x_test, y_test, batch_size=batch_size)

    # 使用 model.to_json() 替换 model.to_yaml()
    json_string = model.to_json()
    with open('/content/drive/MyDrive/iss/NLP Project/SentimentAnalysis-master/model/lstm.json', 'w') as outfile:
        outfile.write(json_string)
    # 继续保存你的模型权重
    model.save_weights('/content/drive/MyDrive/iss/NLP Project/SentimentAnalysis-master/model/lstm.h5')
    print('Test score:', score)

print ('Setting up Arrays for Keras Embedding Layer...')
n_symbols,embedding_weights,x_train,y_train,x_test,y_test=get_data(index_dict, word_vectors,combined,y)
print ("x_train.shape and y_train.shape:")
print (x_train.shape,y_train.shape)
train_lstm(n_symbols,embedding_weights,x_train,y_train,x_test,y_test)

Setting up Arrays for Keras Embedding Layer...
x_train.shape and y_train.shape:
(16870, 100) (16870, 3)
Defining a Simple Keras Model...
Compiling the Model...
Train...
Epoch 1/4
528/528 [==============================] - 56s 101ms/step - loss: 0.7553 - accuracy: 0.6780
Epoch 2/4
528/528 [==============================] - 55s 105ms/step - loss: 0.3823 - accuracy: 0.8780
Epoch 3/4
528/528 [==============================] - 55s 105ms/step - loss: 0.3458 - accuracy: 0.8947
Epoch 4/4
528/528 [==============================] - 54s 103ms/step - loss: 0.2545 - accuracy: 0.9224
Evaluate...
132/132 [==============================] - 3s 20ms/step - loss: 0.2943 - accuracy: 0.9059
Test score: [0.29425284266471863, 0.905879557132721]


In [ ]:
"""
预测
"""
import jieba
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence


from keras.models import model_from_json
np.random.seed(1337)  # For Reproducibility
import sys
sys.setrecursionlimit(1000000)

# define parameters
maxlen = 100

def create_dictionaries(model=None, combined=None):
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        # 使用 model.wv.key_to_index 替代 model.vocab.keys()
        gensim_dict.doc2bow(model.wv.key_to_index.keys(),
                            allow_update=True)
        # 由于 model.wv.key_to_index 已经是 {word: index} 的映射，所以可以直接使用
        w2indx = {word: index + 1 for word, index in model.wv.key_to_index.items()}  # 词语的索引
        w2vec = {word: model.wv[word] for word in w2indx.keys()}  # 词语的词向量

        def parse_dataset(combined):  # 闭包-->临时使用
            data = []
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0)  # 词频小于10的词语索引为0
                data.append(new_txt)
            return data  # word => index
        combined = parse_dataset(combined)
        combined = sequence.pad_sequences(combined, maxlen=maxlen)  # 对句子进行填充
        return w2indx, w2vec, combined
    else:
        print('No data provided...')


def input_transform(string):
    words=jieba.lcut(string)
    words=np.array(words).reshape(1,-1)
    model=Word2Vec.load('/content/drive/MyDrive/iss/NLP Project/SentimentAnalysis-master/model/Word2vec_model.pkl')
    _,_,combined=create_dictionaries(model,words)
    return combined


def lstm_predict(string):
    print('loading model......')

    with open('/content/drive/MyDrive/iss/NLP Project/SentimentAnalysis-master/model/lstm.json', 'r') as f:
        json_string = f.read()
    model = model_from_json(json_string)

    print('loading weights......')
    model.load_weights('/content/drive/MyDrive/iss/NLP Project/SentimentAnalysis-master/model/lstm.h5')
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    data = input_transform(string)
    data = data.reshape(1, -1)  # 注意这里需要用赋值的方式来更新 data 的形状
    # print(data)

    # 使用 predict 方法来替代 predict_classes 方法
    result = model.predict(data)
    predicted_class = np.argmax(result, axis=1)  # 获取最高概率的类别索引
    print(predicted_class)  # 输出预测的类别索引，比如 [1]

    if predicted_class[0] == 1:
        print(string, 'positive')
    elif predicted_class[0] == 0:
        print(string, 'neutral')
    else:
        print(string, 'negative')

In [ ]:
# string='酒店的环境非常好，价格也便宜，值得推荐'
# string='手机质量太差了，傻逼店家，赚黑心钱，以后再也不会买了'
# string = "这是我看过文字写得很糟糕的书，因为买了，还是耐着性子看完了，但是总体来说不好，文字、内容、结构都不好"
# string = "虽说是职场指导书，但是写的有点干涩，我读一半就看不下去了！"
# string = "书的质量还好，但是内容实在没意思。本以为会侧重心理方面的分析，但实际上是婚外恋内容。"
# string = "不是太好"
# string = "不错不错"
string = "你真的太好了啊"
# string = "真的一般，没什么可以学习的"

lstm_predict(string)

loading model......
loading weights......


1/1 [==============================] - 0s 257ms/step
[1]
你真的太好了啊 positive
